In [2]:
import random
import torch.nn as nn
import torch
import pickle
import pandas as pd
from pandas import Series, DataFrame
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, matthews_corrcoef, f1_score, precision_score, recall_score
import numpy as np
import torch.optim as optim
folder = "/data/AIpep-clean/"
import matplotlib.pyplot as plt
from vocabulary import Vocabulary
from datasetbioactivity import Dataset, collate_fn
from models import Classifier
import os

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Load data

In [3]:
df = pd.read_pickle(folder + "pickles/DAASP_RNN_dataset.plk")

df_training = df[df["Set"]=="training"]
df_test = df[df["Set"]=="test"]

vocabulary = Vocabulary.get_vocabulary_from_sequences(df_training.Sequence.values)

if torch.cuda.is_available():
    device = "cuda" 
else:
    device = "cpu" 

# Define helper functions

In [3]:
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return category_i

def nan_equal(a,b):
    try:
        np.testing.assert_equal(a,b)
    except AssertionError:
        return False
    return True

def models_are_equal(model1, model2):
    model1.vocabulary == model2.vocabulary
    model1.hidden_size == model2.hidden_size
    for a,b in zip(model1.model.parameters(), model2.model.parameters()):
        if nan_equal(a.detach().numpy(), b.detach().numpy()) == True:
            print("true")

# Define model

In [4]:
def training(model, test_dataloader, training_dataloader, n_epoch, optimizer, filename):
    
    roc_training = []
    roc_test = []
    
    for e in range(1, n_epoch + 1):
        for i_batch, sample_batched in enumerate(training_dataloader):
            seq_batched = sample_batched[0][0].to(model.device, non_blocking=True)
            seq_lengths = sample_batched[0][1].to(model.device, non_blocking=True)
            cat_batched = sample_batched[1].to(model.device, non_blocking=True)

            output = model.evaluate(seq_batched, seq_lengths)

            loss = criterion(output, cat_batched)

            optimizer.zero_grad()
            loss.backward()  
            torch.nn.utils.clip_grad_value_(model.model.parameters(), 2)
            optimizer.step()

        model.save(filename.format(e))
        
        def _evaluate_ROC(data_loader):
            cat_list = []
            out_list = []

            with torch.no_grad():
                for i_batch, sample_batched in enumerate(data_loader):    
                    seq_batched = sample_batched[0][0].to(model.device, non_blocking=True)
                    seq_lengths = sample_batched[0][1].to(model.device, non_blocking=True)
                    
                    cat_list += sample_batched[1].to("cpu", non_blocking=True)
                    out_list += torch.exp(model.evaluate(seq_batched, seq_lengths))[: ,1].to("cpu", non_blocking=True)

                cat_list = torch.stack(cat_list)
                out_list = torch.stack(out_list)

                roc = roc_auc_score(cat_list.cpu().numpy().astype(int), out_list.cpu().numpy())
            return roc
        
        roc_tr = _evaluate_ROC(training_dataloader)
        roc_te = _evaluate_ROC(test_dataloader)
        roc_training.append(roc_tr)
        roc_test.append(roc_te)
        print("epoch: " + str(e))
        print("roc auc training: " + str(roc_tr))
        print("roc auc test: " + str(roc_te))
        if roc_training == 1.0:
            break
        
    return model, optimizer, roc_training, roc_test
        

In [5]:
learning_rate = 0.01
momentum = 0.9
batch_size = 20
n_epoch = 150
criterion = nn.NLLLoss()

# Hyper parameters optimization

In [ ]:
n_embeddings  = [2, 21, 42, 100]
n_hiddens = [50, 100, 200, 300, 400]
n_layerss = [1, 2,3]
    
if not os.path.exists(folder+"pickles/classifier_hyperparameter_optimization_results.pkl"):
    df_opt = df_training.copy()
    # create an evaluation/training set only from the training set
    # assign to training or evaluation set
    df_opt["Set2"] = "eval"
    training_ = df_opt.sample(frac=0.75, random_state=0)
    df_opt.loc[training_.index, "Set2"] = "training"

    df_training = df_opt[df_opt["Set2"]=="training"]
    df_eval = df_opt[df_opt["Set2"]=="eval"]

    training_dataset = Dataset(df_training, vocabulary)
    eval_dataset = Dataset(df_eval, vocabulary)

    training_dict = {}
    for  n_embedding in n_embeddings:
        for n_hidden in n_hiddens:
            for n_layers in n_layerss:

                if "em{}_hi{}_la{}".format(n_embedding, n_hidden, n_layers) in training_dict:
                    continue

                print(f"dimensions of embedding {n_embedding}, dimensions of hidden {n_hidden}, number of layers {n_layers}")
                model = Classifier(n_embedding, n_hidden, n_layers, vocabulary)
                model.to(device)
                optimizer = optim.SGD(model.model.parameters(), lr = learning_rate, momentum=momentum)
                training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, collate_fn = collate_fn, drop_last=True, pin_memory=True, num_workers=4)
                test_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size, shuffle=True, collate_fn = collate_fn, drop_last=True, pin_memory=True, num_workers=4)
                filename = folder+"models/RNN-classifier/em{}_hi{}_la{}_ep{{}}".format(n_embedding, n_hidden, n_layers)
                model, optimizer, roc_training, roc_test = training(model, test_dataloader, training_dataloader, n_epoch, optimizer, filename)
                training_dict["em{}_hi{}_la{}".format(n_embedding, n_hidden, n_layers)] = [roc_training, roc_test]
                print(f"maximum roc auc for test set {max(roc_test)}")

    with open(folder+"pickles/classifier_hyperparameter_optimization_results.pkl","bw") as fd:
        pickle.dump(training_dict, fd)
else:
    with open(folder+"pickles/classifier_hyperparameter_optimization_results.pkl",'rb') as fd:
        training_dict = pickle.load(fd)

dimensions of embedding 2, dimensions of hidden 50, number of layers 1
epoch: 1
roc auc training: 0.5481353585102527
roc auc test: 0.5558467825935451
epoch: 2
roc auc training: 0.5589495704718545
roc auc test: 0.5673727524213434
epoch: 3
roc auc training: 0.5692983445414421
roc auc test: 0.5737823208786658
epoch: 4
roc auc training: 0.5778282302582335
roc auc test: 0.5801229971302353
epoch: 5
roc auc training: 0.5901009827048896
roc auc test: 0.5864464535743279
epoch: 6
roc auc training: 0.6013038766832755
roc auc test: 0.5966517284965482
epoch: 7
roc auc training: 0.6158598916781532
roc auc test: 0.6060514230806625
epoch: 8
roc auc training: 0.6467439694616739
roc auc test: 0.630977589052235
epoch: 9
roc auc training: 0.6832945364556292
roc auc test: 0.6722976171988925
epoch: 10
roc auc training: 0.7043922665433644
roc auc test: 0.6957373149705317
epoch: 11
roc auc training: 0.7140952871838204
roc auc test: 0.7011429018748288
epoch: 12
roc auc training: 0.7193101938813571
roc auc test

# Optimized hyper parameters

In [ ]:
max_test = 0
for k,v in training_dict.items():
    if max(v[1]) > max_test:
        max_test = max(v[1])
        best = k
best = best.split("_")
n_embedding = int(best[0].replace("em", ""))
n_hidden = int(best[1].replace("hi", ""))
n_layers = int(best[2].replace("la", ""))

# Training

In [ ]:
training_dataset = Dataset(df_training, vocabulary)
test_dataset = Dataset(df_test, vocabulary)

print(f"dimensions of embedding {n_embedding}, dimensions of hidden {n_hidden}, number of layers {n_layers}")

model = Classifier(n_embedding, n_hidden, n_layers, vocabulary)
model.to(device)

optimizer = optim.SGD(model.model.parameters(), lr = learning_rate, momentum=momentum)
training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, collate_fn = collate_fn, drop_last=True, pin_memory=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn = collate_fn, drop_last=True, pin_memory=True, num_workers=4)

filename = folder + "/models/RNN-classifier/em{}_hi{}_la{}_ep{{}}".format(n_embedding, n_hidden, n_layers)
model, optimizer, roc_training, roc_test = training(model, test_dataloader, training_dataloader, n_epoch, optimizer, filename)

print(f"maximum roc auc for test set {max(roc_test)}")

epoch 38
n_embedding 100
n_hidden 400
n_layers 2

In [ ]:
training_dataset = Dataset(df_training, vocabulary)
test_dataset = Dataset(df_test, vocabulary)

roc_test = np.array(roc_test)
epoch = np.argmax(roc_test) + 1

filename = folder + "models/RNN-classifier/em{}_hi{}_la{}_ep{}".format(n_embedding, n_hidden, n_layers, epoch)

model = Classifier.load_from_file(filename)

model.to(device)

training_dataloader_eval = torch.utils.data.DataLoader(training_dataset, batch_size=1, shuffle=False, collate_fn = collate_fn, drop_last=True, pin_memory=True, num_workers=4)
test_dataloader_eval = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn = collate_fn, drop_last=True, pin_memory=True, num_workers=4)

# Evaluation

In [ ]:
def predict(data_loader):
    cat_list = []
    out_list = []

    with torch.no_grad():
        for i_batch, sample_batched in enumerate(data_loader):    
            seq_batched = sample_batched[0][0].to(model.device, non_blocking=True)
            seq_lengths = sample_batched[0][1].to(model.device, non_blocking=True)

            cat_list += sample_batched[1].to("cpu", non_blocking=True)
            out_list += torch.exp(model.evaluate(seq_batched, seq_lengths))[: ,1].to("cpu", non_blocking=True)

        cat_list = torch.stack(cat_list)
        out_list = torch.stack(out_list)
    return cat_list.cpu().numpy().astype(int), out_list.cpu().numpy()

def roc(y_true, y_score):
    fpr, tpr, thresh = roc_curve(y_true, y_score)
    roc = roc_auc_score(y_true, y_score)
    return roc, fpr, tpr
                
def find_threshold(y_true, y_score, alpha = 0.05):
    fpr, tpr, thresh = roc_curve(y_true, y_score)
    for i, fp in enumerate(fpr):
        if fp > alpha:
            return thresh[i-1]
        
def calc_metrics(y_true, y_score, threshold = 0.5):
    y_score = y_score > threshold
    accuracy = accuracy_score(y_true, y_score)
    f1 = f1_score(y_true, y_score)
    mcc = matthews_corrcoef(y_true, y_score)
    precision = precision_score(y_true, y_score)
    recall = recall_score(y_true, y_score)
    return accuracy, f1, mcc, precision, recall
    

In [ ]:
y_true, y_score = predict(test_dataloader_eval)
threshold = find_threshold(y_true, y_score)

### threshold 0.98799103
However, we have kept the treshold from a previuos version of the model 
the treshold is harcoded to 
### threshold 0.99205756 (alpha < 5%)

In [ ]:
threshold = 0.99205756

In [ ]:
accuracy, f1, mcc, precision, recall = calc_metrics(y_true, y_score, threshold)
print(f"accuracy: {accuracy}\nf1 score: {f1}\nmcc: {mcc}\nprecision: {precision}\nrecall: {recall}" )

accuracy: 0.63107202680067
f1 score: 0.4469554300062774
mcc: 0.3513504370839463
precision: 0.8922305764411027
recall: 0.2981574539363484

In [ ]:
accuracy, f1, mcc, precision, recall = calc_metrics(y_true, y_score, 0.5)
print(f"accuracy: {accuracy}\nf1 score: {f1}\nmcc: {mcc}\nprecision: {precision}\nrecall: {recall}" )

accuracy: 0.7621440536013401
f1 score: 0.7711522965350524
mcc: 0.5259204509455355
precision: 0.7430124223602484
recall: 0.8015075376884422

In [ ]:
df["prediction"] = df.Sequence.map(lambda x: model.predict_peptide_sequence(x)[:,1][0])

In [ ]:
Y_true, Y_score = predict(training_dataloader_eval) 
roc_training, fpr_training, tpr_training = roc(Y_true, Y_score)
y_true, y_score = predict(test_dataloader_eval)
roc_test, fpr_test, tpr_test = roc(y_true, y_score)

plt.figure()
name = "RNN classifier"
plt.rcParams.update({'font.size': 15})
lw = 2
plt.plot(fpr_training, tpr_training, color='blue',
         lw=lw, label='Training Set ROC AUC = %0.2f' % roc_training)
plt.plot(fpr_test, tpr_test, color='darkorange',
         lw=lw, label='Test Set ROC AUC = %0.2f' % roc_test)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([-0.005, 1.0])
plt.ylim([-0.005, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(name)
plt.legend(loc="lower right")
plt.tight_layout()
plt.savefig(folder+"plots/RNN-classifier.svg")
plt.savefig("plots/RNN-classifier.svg")
plt.show()

In [ ]:
df.to_pickle(folder + "pickles/DAASP_RNN_dataset_with_prediction.plk")

# Test a larger test set

In [ ]:
def scramble_less(seq):
    lengths = [1,2]
    n = random.choice(lengths)
    seq_list = [seq[i:i+n] for i in range(0, len(seq), n)]
    new_seq = ''
    parts = seq_list
    while len(new_seq) < len(seq):
        part = random.choice(parts)
        parts.remove(part)
        new_seq += part
    return new_seq

def new_inactive(row):
    seq = scramble_less(row["Sequence"])
    cid = "scr_{}".format(row["ID"])
    new_row = {"ID":[cid], "Name":[None], "N terminus":[None], "Sequence":[seq], "C terminus":[None], \
               "targetSpecies":[None], "baumannii":[False], "aureus":[False], "aeruginosa":[False], "activity":[0], "Set":["test"], "prediction":[None]}
    return pd.DataFrame(new_row)

df_more_scrs = []
for i in range(15):
    df_more_scr = df_test.apply(new_inactive, axis=1)
    df_more_scr = pd.concat(df_more_scr.tolist()).reset_index(drop = True)
    df_more_scrs.append(df_more_scr)
    
df_more_scrs__ = pd.concat(df_more_scrs)
df_more_test = pd.concat([df_more_scrs__, df_test]).reset_index(drop = True)
df_more_test = df_more_test.drop_duplicates("Sequence").copy()
df_more_test = df_more_test.reset_index(drop = True).copy()
more_test_dataset = Dataset(df_more_test, vocabulary)
test_dataloader_eval_more = torch.utils.data.DataLoader(more_test_dataset, batch_size=1, shuffle=False, collate_fn = collate_fn, drop_last=True, pin_memory=True, num_workers=0)
y_true_more, y_score_more = predict(test_dataloader_eval_more) 

#same threshold as before
accuracy_more, f1_more, mcc_more, precision_more, recall_more = calc_metrics(y_true_more, y_score_more, threshold)
print(f"accuracy: {accuracy_more}\nf1 score: {f1_more}\nmcc: {mcc_more}\nprecision: {precision_more}\nrecall: {recall_more}")